# Constants

In [53]:
import numpy as np
from sklearn.metrics import make_scorer

DROPPED_LEXICAL_COLUMNS = [
    "Swear",
    "Numbers",
    "Inhibition",
    "Preceptual",
    "Anxiety",
    "Anger",
    "Sadness",
    "Work",
    "Articles",
    "Verbs",
    "Adverbs",
    "Prepositions",
    "Conjunctions",
    "Negations",
]

facial_features = [
    "average_inner_brow_height",
    "average_outer_brow_height",
    "eye_open",
    "inner_lip_height",
    "lip_corner_distance",
    "outer_lip_height",
    "smile",
    "pitch",
    "roll",
    "yaw",
]
stats = ["max", "median", "min", "std", "mean"]
DROPPED_FACIAL_FEATURES = [
    f"{feature}_{stat}" for feature in facial_features for stat in stats
]

ALREADY_NORMALIZED_FEATURES = [
    "average_outer_brow_height_mean",
    "average_inner_brow_height_mean",
    "eye_open_mean",
    "inner_lip_height_mean",
    "inner_lip_height_mean",
    "lip_corner_distance_mean",
    "average_outer_brow_height_std",
    "average_inner_brow_height_std",
    "eye_open_std",
    "outer_lip_height_std",
    "inner_lip_height_std",
    "lip_corner_distance_std",
    "average_outer_brow_height_min",
    "average_inner_brow_height_min",
    "eye_open_min",
    "outer_lip_height_min",
    "inner_lip_height_min",
    "lip_corner_distance_min",
    "average_outer_brow_height_max",
    "average_inner_brow_height_max",
    "eye_open_max",
    "outer_lip_height_max",
    "inner_lip_height_max",
    "lip_corner_distance_max",
    "average_outer_brow_height_median",
    "average_inner_brow_height_median",
    "eye_open_median",
    "outer_lip_height_median",
    "inner_lip_height_median",
    "lip_corner_distance_median",
]  # these are already in [0, 1]

DROPPED_PROSODIC_COLUMNS = []
MUST_KEEP_FEATURES = [
    "pause_duration_avg",
    "average_outer_brow_height_mean",
    "average_inner_brow_height_mean",
    "outer_lip_height_mean",
    "Duration/Filler Words",
]

TARGET_COLUMN = "RecommendHiring"
GROUPS_COLUMN = "cleaned_ids"
INDEX_COLUMN = "participant_id"


def pearson_corr(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0, 1]


SCORING_METRICS = {
    "r2": "r2",
    "mae": "neg_mean_absolute_error",
    "pearson": make_scorer(pearson_corr),  # Pearson Correlation Coefficient
}


MUST_KEEP_FEATURES = [
    # "pause_duration_avg",
    # "average_outer_brow_height_mean",
    # "average_inner_brow_height_mean",
    # "outer_lip_height_mean",
    "Duration/Filler Words",
]

PIPELINE_PARAMS = {
    "feature_selection__threshold": None,
    "svr__C": 0.1,
    "svr__coef0": 1.0,
    "svr__degree": 3,
    "svr__epsilon": 0.5,
    "svr__gamma": 0.1,
    "svr__kernel": "poly",
    "svr__shrinking": True,
    # "feature_selection__threshold": None,
    # "svr__C": 10.0,
    # "svr__coef0": 0.0,
    # "svr__degree": 2,
    # "svr__epsilon": 0.01,
    # "svr__gamma": 0.001,
    # "svr__kernel": "rbf",
    # "svr__shrinking": True,
}
HYPERPARAMETER_TUNING_ENABLED = True

# Data Preprocessing

## Import Datasets

In [54]:
import pandas as pd
import os

datasets_folder_path = "../datasets"

features_df = pd.read_csv(os.path.join(datasets_folder_path, "add.csv"))
features_df = features_df.set_index("participant_id")

labels_df = pd.read_csv(
    os.path.join(datasets_folder_path, "turker_scores_full_interview.csv")
)
labels_df = labels_df.set_index("Participant")
labels_df = labels_df.loc[labels_df["Worker"] == "AGGR"]

features_df.index = features_df.index.str.lower()
labels_df.index = labels_df.index.str.lower()
indexed_combined_df = features_df.join(labels_df[[TARGET_COLUMN]], how="left")

## Drop Unnecessary Columns

In [55]:
combined_df = indexed_combined_df.reset_index(drop=True)
combined_df = combined_df.drop(
    columns=DROPPED_FACIAL_FEATURES + DROPPED_LEXICAL_COLUMNS + DROPPED_PROSODIC_COLUMNS
)

# Model

## Split Data

In [56]:
X = combined_df.drop(columns=[TARGET_COLUMN, GROUPS_COLUMN])
y = combined_df[TARGET_COLUMN]

## Pipeline Creation

In [57]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GroupKFold, cross_val_score, cross_validate
import sys

sys.path.append("..")
from models.domain_aware_selector import DomainAwareSelector

groups_column = combined_df[GROUPS_COLUMN].astype(str).values

lasso_feature_selection_model = LassoCV(
    cv=5,
    random_state=42,
    max_iter=30000,
    alphas=np.logspace(-3, 0, 30),
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="mean")),  # NaN imputation
        ("scaler", StandardScaler()),
        ("feature_selection", SelectFromModel(estimator=lasso_feature_selection_model)),
        # ('feature_selection', DomainAwareSelector(
        #     must_keep_features=MUST_KEEP_FEATURES,
        #     selector=SelectFromModel(lasso_feature_selection_model, max_features=10),
        # )),
        ("svr", SVR(kernel="rbf")),
    ]
)

## Hyperparameter Tuning

In [58]:
from sklearn.model_selection import GridSearchCV, GroupShuffleSplit, RandomizedSearchCV


param_grid = {
    # Feature selection parameters
    "feature_selection__threshold": [None, "median"],
    # 'feature_selection__selector__threshold': [None, 'median'],
    # SVR parameters
    "svr__C": np.logspace(-2, 2, 5),  # [0.01, 0.1, 1, 10, 100]
    "svr__gamma": ["scale", "auto"] + list(np.logspace(-3, 1, 5)),  # More gamma options
    "svr__epsilon": [0.01, 0.1, 0.5],
    "svr__kernel": ["rbf", "poly"],  # Added poly kernel
    "svr__degree": [2, 3],  # Only used for poly kernel
    "svr__coef0": [0.0, 1.0],  # Important for poly kernel
    # "svr__shrinking": [
    #     True,
    #     False,
    # ],  # the difference may be subtle—often the default (shrinking=True) works well
}

if HYPERPARAMETER_TUNING_ENABLED:
    mc_cv_tuning = GroupShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
    
    random_search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_grid,
        cv=mc_cv_tuning,  # Use Monte Carlo splits
        scoring="r2",
        verbose=1,
        n_jobs=-1,
        n_iter=20,  # Number of random samples to test
        random_state=42,
    )
    
    random_search.fit(X, y, groups=groups_column)  # Groups ensure participant-level splits
    
    print("Best hyperparameters:", random_search.best_params_)
    print(f"Best R² score from RandomizedSearchCV: {random_search.best_score_:.4f}")

Fitting 100 folds for each of 20 candidates, totalling 2000 fits
Best hyperparameters: {'svr__kernel': 'rbf', 'svr__gamma': 0.01, 'svr__epsilon': 0.1, 'svr__degree': 3, 'svr__coef0': 1.0, 'svr__C': 1.0, 'feature_selection__threshold': None}
Best R² score from RandomizedSearchCV: 0.1906


## Store Trained Model

In [64]:
from sklearn import clone


if HYPERPARAMETER_TUNING_ENABLED:
    pipeline = random_search.best_estimator_
else:
    pipeline.set_params(**PIPELINE_PARAMS)
model = clone(pipeline)
model.fit(X, y)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('feature_selection',
                 SelectFromModel(estimator=LassoCV(alphas=array([0.001     , 0.00126896, 0.00161026, 0.00204336, 0.00259294,
       0.00329034, 0.00417532, 0.00529832, 0.00672336, 0.00853168,
       0.01082637, 0.01373824, 0.01743329, 0.02212216, 0.02807216,
       0.03562248, 0.04520354, 0.05736153, 0.07278954, 0.09236709,
       0.11721023, 0.14873521, 0.18873918, 0.23950266, 0.30391954,
       0.38566204, 0.48939009, 0.62101694, 0.78804628, 1.        ]),
                                                   cv=5, max_iter=30000,
                                                   random_state=42))),
                ('svr', SVR(coef0=1.0, gamma=0.01))])

## Feature Selection Results

In [65]:
selected_mask = model.named_steps["feature_selection"].get_support()

selected_features_bool_mask = model.named_steps[
    "feature_selection"
].get_support()  # get_support returns a boolean mask
selected_feature_names = X.columns[selected_features_bool_mask]
unselected_feature_names = X.columns[~selected_features_bool_mask]
print(f"Number of Selected features ({len(selected_feature_names)}):")
print(f"Selected features ({selected_feature_names}):")
print(f"Unselected features :{unselected_feature_names}):")

Number of Selected features (12):
Selected features (Index(['f0_sd', 'intensity_mean', 'f2_mean', 'f3_mean', 'f3_sd', 'f2_f1_mean',
       'percent_unvoiced', 'percent_breaks', 'Duration/Total Words', 'They',
       'NegEmotion', 'Cognitive'],
      dtype='object')):
Unselected features :Index(['f0_mean', 'f0_min', 'f0_max', 'f0_range', 'intensity_min',
       'intensity_max', 'intensity_range', 'intensity_sd', 'f1_mean', 'f1_sd',
       'f2_sd', 'f3_f1_mean', 'f2_f1_sd', 'f3_f1_sd', 'jitter', 'shimmer',
       'pause_duration_max', 'pause_duration_avg', 'duration', 'Total Words',
       'Unique Words', 'Filler Words', 'Audio Duration (s)',
       'Duration/Unique Words', 'Duration/Filler Words', 'Individual', 'We',
       'Non-Fluences', 'PosEmotion', 'Relativity', 'Quantifiers'],
      dtype='object')):


# Monte Carlo Cross Validation

In [67]:
from sklearn import clone
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import r2_score
import numpy as np
from scipy.stats import pearsonr

N_TRIALS = 1000
TEST_SIZE = 0.2  # 80% training, 20% testing
r2_scores = []
pearson_scores = []

# Use GroupShuffleSplit to ensure that all data from the same participant (group) is kept together.
gss = GroupShuffleSplit(n_splits=N_TRIALS, test_size=TEST_SIZE, random_state=42)

for train_idx, test_idx in gss.split(X, y, groups=groups_column):  # runs n_splits times
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Clone the pipeline to ensure each trial is independent
    model = clone(pipeline) #  creates a new instance of the pipeline with the same hyperparameters, but without the fitted parameters. This new instance starts fresh for each Monte Carlo iteration.

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Compute R² score
    r2_scores.append(r2_score(y_test, y_pred))

    # Compute Pearson correlation coefficient
    pearson_coef, _ = pearsonr(y_test, y_pred)
    pearson_scores.append(pearson_coef)


# Report the average performance and variability
print(
    f"Mean R² Score over {N_TRIALS} trials: {np.mean(r2_scores):.2f} (±{np.std(r2_scores):.2f})"
)
print(
    f"Mean Pearson Correlation over {N_TRIALS} trials: {np.mean(pearson_scores):.2f} (±{np.std(pearson_scores):.2f})"
)

Mean R² Score over 1000 trials: 0.18 (±0.24)
Mean Pearson Correlation over 1000 trials: 0.52 (±0.16)
